Benchmarks using four methods in python

1. Vanilla Python
2. Vectorized Python
3. JIT
4. PyO3 (Rust)

In [42]:
import numpy as np
from numba import jit, prange
import random
import time
import ctypes

In [38]:
lim: int = 1000000000
PI: float = np.pi

# Vanilla

In [26]:
valid = 0

for i in range(lim):
    if (random.uniform(0, 1)**2 + random.uniform(0, 1)**2) <= 1:
        valid += 1

aprox = valid*4/lim
err = (1-aprox/PI)*100

print(f"Aproximation of Pi: {aprox}\n")
print(f"Real  valor  of Pi: {PI}\n")
print(f"Error: {err}%")

Aproximation of Pi: 3.14178756

Real  valor  of Pi: 3.141592653589793

Error: -0.006204063724934983%


# Vectorized

In [ ]:
#USAGE OF 16Gb RAM

def vectorized(n:int)->float:
    point = np.random.rand(n)**2 + np.random.rand(n)**2
    aprox = np.sum(point <= 1)*4/n

    return aprox

# JIT version (C & multithreading)

In [40]:
@jit(nopython=True)
def monte_carlo_c(n:int) -> float:
    valid = 0
    for i in range(n):
        x = random.random()
        y = random.random()
        if x**2 + y**2 <= 1:
            valid += 1
    return valid*4/n

start = time.time()
aprox = monte_carlo_c(lim)
end = time.time()

print(f"Arpximation Pi: {aprox} obtained in {end-start}")
print(f"Valor real  Pi: {PI}")

print(f"\n Error de {(1-aprox/PI)*100}%")


Arpximation Pi: 3.141550444 obtained in 44.16309428215027
Valor real  Pi: 3.141592653589793

 Error de 0.0013435729722899303%


In [41]:
@jit(parallel=True)
def monte_carlo_p(n:int) -> float:
    valid = 0
    for i in prange(n):
        x = random.random()
        y = random.random()
        if x**2 + y**2 <= 1:
            valid += 1
    return valid*4/n

start = time.time()
aprox = monte_carlo_p(lim)
end = time.time()

print(f"Arpximation Pi: {aprox} obtained in {end-start}")
print(f"Valor real  Pi: {PI}")

print(f"\n Error de {(1-aprox/PI)*100}%")

Arpximation Pi: 3.141627448 obtained in 6.499444007873535
Valor real  Pi: 3.141592653589793

 Error de -0.0011075404752869389%


# Using extern binaries (Rust)

In [49]:
lib = ctypes.CDLL('./libmontecarlo.so')

lib.monte_carlo_rust.argtypes = (ctypes.c_int, )
lib.monte_carlo_rust.restype = (ctypes.c_double)

start = time.time()
aprox = lib.monte_carlo_rust(lim)
end = time.time()

#print(f"Arpximation Pi: {aprox} obtained in {end-start}")
#print(f"Valor real  Pi: {PI}")

#print(f"\n Error de {(1-aprox/PI)*100}%")

Dentro del circulo hubo 785385571-78.53855717853855%
Aproximacion de pi: 3.1415422871415424
Valor real de pi:   3.1415927
Error de 0.00001603213841017137%

Calculado en 39.601517938s
